In [ ]:
#pip install python-google-places

In [18]:
import pandas as pd
import os
from dotenv import load_dotenv
import googlemaps
from geopy import distance
import folium
import numpy as np
import urllib.request
import json
import requests
from googleplaces import GooglePlaces, types, lang

In [19]:
data = pd.read_csv('lista_final.csv',sep='|')

In [20]:
load_dotenv()

True

In [21]:
API_key=os.getenv('API')

In [22]:
gmaps_key = googlemaps.Client(key=API_key)

In [23]:
google_places = GooglePlaces(API_key)

In [24]:
def get_coord(string):
    geocode_result = gmaps_key.geocode(string)
    try:
        lat = geocode_result[0]['geometry']['location']['lat']
        lon = geocode_result[0]['geometry']['location']['lng']
        return [lat,lon]
    except:
        return [None,None]

In [25]:
def get_distance(pointx, pointy):
    return distance.distance(pointx,pointy)

In [26]:
get_distance([-39, 99], [-40, 98])

Distance(140.44294774014736)

In [27]:
url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters"

In [28]:
def rentas():
    rango_precios = {'1':'5000-10000','2':'10000-15000','3':'15000-20000','4':'20000-25000','5':'25000-30000','6':'30000-35000','7':'35000-40000','8':'40000-45000','9':'45000-50000','10':'>50000'}
    dummy = data.copy()
    
    indice_precios = rango_precios[input(f'Selecciona un rango de precios: {str(rango_precios)}')].split('-')
    
    dummy = dummy[dummy['Price(MX$)'].between(int(indice_precios[0]), int(indice_precios[1]))]
    
    indice_hab = int(input((f'Selecciona número de habitantes: ')))
    dummy = dummy[dummy['Bedrooms'] == indice_hab]

    coord_trab = input(f'¿Cuál es tu ubicacion? : ')

    ubicacion = get_coord(coord_trab)
    dummy['Distance'] = dummy['Coordinates'].str.split(',').apply(\
                        lambda x:get_distance(x, ubicacion))
    
    #hobby = input(f'¿Cuál es tu hobby? : ')
    
    dummy = dummy[dummy['Distance']<=5]
    
    dummy['Phone Number'] = np.random.randint(100000000, 10000000000, dummy.shape[0])
    
    dummy.drop_duplicates(subset='Address', inplace=True)
    
    return  dummy, ubicacion

In [29]:
test, ubi = rentas()

Selecciona un rango de precios: {'1': '5000-10000', '2': '10000-15000', '3': '15000-20000', '4': '20000-25000', '5': '25000-30000', '6': '30000-35000', '7': '35000-40000', '8': '40000-45000', '9': '45000-50000', '10': '>50000'}2
Selecciona número de habitantes: 2
¿Cuál es tu ubicacion? : tonala 10


In [30]:
ubi

[19.4212752, -99.1631294]

In [31]:
test

,Address,Price(MX$),Bedrooms,Bathrooms,Size(m2),Latitud,Longitud,Coordinates,Distance,Phone Number
38,"Centro (área 1), Cuauhtémoc, Distrito Federa...",14000,2,2.0,60,19.430268,-99.137314,"19.430267800000003,-99.1373136",2.8881806728454142 km,730834154
137,"Golfo De San Jorge 37 Tacuba Miguel Hidalgo,...",12000,2,2.0,69,19.454628,-99.186314,"19.454627600000002,-99.18631440000001",4.42255326153396 km,8606343424
152,"Guanábana 10, Nueva Santa Maria, Azcapotzalc...",10500,2,1.0,75,19.459330,-99.169135,"19.4593296,-99.1691353",4.259473900646973 km,7434409708
156,"Lago Zirahuen 98, Colonia Anahuac I Sección,...",11500,2,1.0,58,19.445192,-99.172877,"19.445192300000002,-99.1728766",2.8385483994740905 km,1768470751
186,"R.salido 166, Independencia, Benito Juárez, ...",13950,2,2.0,75,19.382154,-99.150305,"19.3821537,-99.1503048",4.535295606572306 km,4564588195
290,"Rafael Solana 5, Independencia, Benito Juáre...",13200,2,2.0,70,19.381035,-99.145823,"19.3810355,-99.145823",4.811051954645216 km,4256821453
296,"Thomas A Edison 139, San Rafael, Cuauhtémoc,...",15000,2,1.0,65,19.438373,-99.156752,"19.4383727,-99.1567523",2.0076424329775655 km,2976378159
310,"11 De Abril 177 303 A, Ciudad De México, Dis...",12500,2,1.0,120,19.396047,-99.184039,"19.3960468,-99.184039",3.552807435157517 km,1789771590
338,"Peral 26, Santa Maria La Ribera, Cuauhtémoc,...",12500,2,2.0,67,19.454342,-99.162272,"19.4543423,-99.1622723",3.661546915267793 km,6906789442
388,"Av Cuauhtemoc 514, Narvarte, Benito Juárez, ...",15000,2,2.0,62,19.399603,-99.155501,"19.3996034,-99.1555014",2.5292497803233265 km,4332807835


In [32]:
all_places = []

In [33]:
for address in test.Address:
    query_result = google_places.nearby_search(
    location= address, keyword='Restaurants',
    radius=1000, types=[types.TYPE_RESTAURANT])
    all_places= all_places+query_result.places

In [51]:
#all_places

In [20]:
test.Address.iloc[0]

'  Calzada San Simon No. 82, San Simon Ticumac, Benito Juárez, Distrito Federal (cdmx)'

In [34]:
pd.DataFrame(query_result.places)

,0
0,"<Place name=""Bellini"", lat=19.3945055, lng=-99..."
1,"<Place name=""Suntory"", lat=19.3925682, lng=-99..."
2,"<Place name=""Nagaoka"", lat=19.3915959, lng=-99..."
3,"<Place name=""Marina Bistro"", lat=19.3810327, l..."
4,"<Place name=""El Afán"", lat=19.3808647, lng=-99..."
5,"<Place name=""IHOP"", lat=19.3945081, lng=-99.17..."
6,"<Place name=""D' Polak"", lat=19.3897217, lng=-9..."
7,"<Place name=""La Vid Argentina"", lat=19.3844962..."
8,"<Place name=""Piloncillo y Cascabel"", lat=19.38..."
9,"<Place name=""La Strega"", lat=19.3953784, lng=-..."


In [35]:
places = {('place'+ str(v)): [all_places[v].name,all_places[v]._geo_location['lat'],all_places[v]._geo_location['lng']] for v in range(len(all_places))}

In [36]:
places = pd.DataFrame(places).T


In [37]:
places

,0,1,2
place0,Azul Historico,19.4328594,-99.13621999999999
place1,Cafe De Tacuba,19.4357362,-99.1376147
place2,Catedral,19.4358258,-99.1331033
place3,Danubio,19.4310795,-99.14099110000001
place4,La India,19.4300597,-99.13880449999999
place5,La Terraza,19.4329724,-99.13435199999999
place6,Miralto,19.4338792,-99.14062369999999
place7,El Balcón del Zócalo,19.433891,-99.134311
place8,Centro Castellano,19.4311366,-99.1400237
place9,La Pagoda Cafeteria,19.4347587,-99.1395345


In [38]:
query_result.places[0]._geo_location

{'lat': Decimal('19.3945055'), 'lng': Decimal('-99.1737931')}

In [39]:
query_result

<GooglePlacesSearchResult with 20 result(s)>

In [40]:
#test['Phone Number'] = np.random.randint(100000000, 100000000000, test.shape[0])

In [41]:
test.head()

,Address,Price(MX$),Bedrooms,Bathrooms,Size(m2),Latitud,Longitud,Coordinates,Distance,Phone Number
38,"Centro (área 1), Cuauhtémoc, Distrito Federa...",14000,2,2.0,60,19.430268,-99.137314,"19.430267800000003,-99.1373136",2.8881806728454142 km,730834154
137,"Golfo De San Jorge 37 Tacuba Miguel Hidalgo,...",12000,2,2.0,69,19.454628,-99.186314,"19.454627600000002,-99.18631440000001",4.42255326153396 km,8606343424
152,"Guanábana 10, Nueva Santa Maria, Azcapotzalc...",10500,2,1.0,75,19.459330,-99.169135,"19.4593296,-99.1691353",4.259473900646973 km,7434409708
156,"Lago Zirahuen 98, Colonia Anahuac I Sección,...",11500,2,1.0,58,19.445192,-99.172877,"19.445192300000002,-99.1728766",2.8385483994740905 km,1768470751
186,"R.salido 166, Independencia, Benito Juárez, ...",13950,2,2.0,75,19.382154,-99.150305,"19.3821537,-99.1503048",4.535295606572306 km,4564588195


depas = rentas()

depas

data['Distance'] = data['Coordinates'].apply(lambda x: get_distance(x.split(",")))

data.sort_values(by='Distance').head()

mapa = folium.Map(
    location=ubi,
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'

for index, row in data.head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row['Address'],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa)

mapa

In [29]:
#test.drop_duplicates(subset='Address', inplace=True)

In [52]:
#test.sort_values(by='Distance').head(10)

In [49]:
mapa2 = folium.Map(
    location=ubi,
    zoom_start=12,
    tiles='openstreetmap')
tooltip = 'Click me!'


for index, row in test.sort_values(by='Distance').head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row[['Address','Bedrooms','Price(MX$)','Bathrooms','Phone Number']],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa2)
    
folium.Marker(ubi, icon=folium.Icon(color='green',icon='home')).add_to(mapa2)
  
for index, row in places.iterrows():
    folium.Marker([row.iloc[1], row.iloc[2]],                 
    radius=500,  
    popup=row[0],              
    icon=folium.Icon(color='blue',icon='info-sign')).add_to(mapa2)

mapa2

In [50]:
mapa2.save('mapa_final.html')